# Intelligent Realestate Investing 

In [1]:
#https://rapidapi.com/apidojo/api/realtor
import pandas as pd
import requests
import json #helps manipulate json
import csv # helps with csv files
from bs4 import BeautifulSoup
!pip install pyzillow
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import shutil
# for web driver down below
email = 'blaketindol@gmail.com'
passwordz = 'SWimming178!!' 
!pip install folium
import folium # look in china
import warnings
warnings.filterwarnings('ignore')

#https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Carino+Ct&city=Paw+Paw&state=MI&zip=49079-9661&freeForm=

You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [3]:
# Function using the realator api to grabe houses that are for sale!!

def api_property_list_for_sale(city, state, prop_type, limit=200):

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {"city":city,"limit":"200","offset":"0","state_code":state,"sort":"relevance","prop_type":prop_type}

    headers = {
        'x-rapidapi-key': "3098984b99msh9a120a8ade10e53p1ebdd1jsn7703d6e155d0",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json() # json format

In [4]:
# Once pick state and home type throw it through this function to get the proper readable format
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [5]:
def make_house_data_clean(df_properties_for_sale_raw):
    df_properties_for_sale_raw = pd.DataFrame(df_properties_for_sale_raw)# transfer into dataframe
    df_pol_ps = df_properties_for_sale_raw['lot_size'].apply(pd.Series) # make this int column non interable into series then 
    df_properties_for_sale_raw['size'] = df_pol_ps['size'] # then make it into column 

    # pick out only the columns we want  and rename them 
    list2 = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'size']
    result = df_properties_for_sale_raw[list2]
    result.columns = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'lot_size']
    return result

In [6]:
def get_address_and_buildingsize(result):
    address = pd.json_normalize(result['address'])
    building_size = pd.json_normalize(result['building_size'].dropna())
    building_size.columns =['building_size','building_inside_units']
    result2 = pd.concat([address,building_size], axis=1)
    result3 = pd.concat([df,result2], axis=1)
    return result3


In [7]:
# Variables 
city = "cape coral"
state = "FL"
prop_type = "single_family"

property_list_for_sale_response = api_property_list_for_sale(
                                                             city=city, 
                                                             state=state, 
                                                             prop_type=prop_type,
                                                             limit=200)

In [8]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)


In [9]:
df = make_house_data_clean(df_properties_for_sale_raw)
#list(df.columns)
#df

In [10]:
df['price'].max() # 1949000
df['price'].min() # 34900
df['price'].mean() # 404515.635


492387.24

# Vizualize the points on a map

In [12]:
# expand the address and building size columns 
var2 = get_address_and_buildingsize(df)
var2

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,...,county,fips_code,county_needed_for_uniq,lat,lon,neighborhood_name,time_zone,is_approximate,building_size,building_inside_units
0,M6996370296,2932507847,"{'city': 'Cape Coral', 'line': '4337 SW 1st Pl...",for_sale,439000,2,2,3,"{'size': 1952, 'units': 'sqft'}",2021-08-02T07:18:55Z,...,Lee,12071,False,26.570799,-81.976787,Pelican,NaN,NaN,1952.0,sqft
1,M6142782511,2932503820,"{'city': 'Cape Coral', 'line': '2122 SW 52nd S...",for_sale,599000,2,3,3,"{'size': 2474, 'units': 'sqft'}",2021-08-01T22:52:04Z,...,Lee,12071,False,26.555158,-82.018903,Pelican,NaN,NaN,2474.0,sqft
2,M6066152361,2932503672,"{'city': 'Cape Coral', 'line': '122 SE 10th Te...",for_sale,459000,2,2,3,"{'size': 2196, 'units': 'sqft'}",2021-08-01T21:50:38Z,...,Lee,12071,False,26.633142,-81.972717,Hancock,NaN,NaN,2196.0,sqft
3,M5134366107,2932503348,"{'city': 'Cape Coral', 'line': '4102 Palm Tree...",for_sale,239900,2,2,2,"{'size': 1500, 'units': 'sqft'}",2021-08-01T21:15:44Z,...,Lee,12071,False,26.575923,-81.966890,Caloosahatchee,NaN,NaN,1500.0,sqft
4,M6087382576,2932502197,"{'city': 'Cape Coral', 'line': '2044 NE 20th T...",for_sale,224900,2,2,3,"{'size': 1800, 'units': 'sqft'}",2021-08-01T19:27:26Z,...,Lee,12071,False,26.690524,-81.931518,Diplomat,America/New_York,NaN,1800.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M5606238354,2932179956,"{'city': 'Cape Coral', 'line': '2013 NE 19th T...",for_sale,269000,2,2,3,"{'size': 1350, 'units': 'sqft'}",2021-07-27T14:26:56Z,...,Lee,12071,False,26.690300,-81.933495,Diplomat,America/New_York,NaN,2374.0,sqft
196,M5988491574,2932178590,"{'city': 'Cape Coral', 'line': '2906 NW 11th S...",for_sale,478900,2,3,3,"{'size': 2374, 'units': 'sqft'}",2021-08-01T23:04:51Z,...,Lee,12071,False,26.672454,-82.034916,Burnt Store,NaN,NaN,1718.0,sqft
197,M6429443512,2932154313,"{'city': 'Cape Coral', 'line': '313 NW 16th Pl...",for_sale,279700,2,2,3,"{'size': 1718, 'units': 'sqft'}",2021-07-26T15:41:09Z,...,Lee,12071,False,26.658348,-82.007905,Mariner,NaN,NaN,2182.0,sqft
198,M5891264730,2932152767,"{'city': 'Cape Coral', 'line': '2918 SW 38th T...",for_sale,750000,3,3,3,"{'size': 2182, 'units': 'sqft'}",2021-07-24T13:35:36Z,...,Lee,12071,False,26.580697,-82.035870,Pelican,NaN,NaN,2470.0,sqft


In [13]:
lon = list(var2["lon"]) # make longitude adn lattitude into list
lat = list(var2["lat"])
address = list(var2['line'])
price = list(var2["price"]) # This normally needs to be a string!!!!!!!!!

# Color changing function 
def color_producer(prices):
    if prices <= 20000:
        return 'lightgreen'
    elif 20001 <= prices < 75000:
        return 'green'
    elif 75001 <= prices < 100000:
        return 'darkgreen'
    elif 100001 <= prices < 200000:
        return 'lightblue'
    elif 200001 <= prices < 250000:
        return 'blue'
    elif 250001 <= prices < 300000:
        return 'darkblue'
    elif 300001 <= prices < 350000:
        return 'orange'
    elif 350001 <= prices < 400000:
        return 'lightred'
    else: 
        return 'red'
# 'pink', 'darkblue', 'red', 'purple', 'darkred', 'cadetblue', 'lightred', 'green', 'darkgreen', 'beige', 'lightgreen', 'orange', 'white', 'gray', 'darkpurple', 'blue', 'lightgray', 'black', 'lightblue'}.
# initialize map
map = folium.Map(location = [38.28,-99.09],zoom_start = 6,TileLayer="Mapbox Bright")
#make map fg variable 
fg = folium.FeatureGroup(name="My map")

for lt, ln, pr , adr in zip(lat,lon,price,address):
        fg.add_child(folium.Marker(location =[lt,ln],popup=str(pr) + " dollars" + str(adr) ,icon =folium.Icon(color=color_producer(pr))))

map.add_child(fg)
map.save("Map10.html")
map

In [14]:
street = var2['line'][1] 
street=street[4:]
street = street.strip()
street

'SW 52nd St'

# Loop through dataframe street name using selenium and Melisa.com

In [16]:
for i in range(0,len(var2)): 
    try:
        street = var2['line'][i]                                 # get the street name
        #result = ''.join([i for i in street if not i.isdigit()]) # search for digits and replace them when searching for the street
        result=street[4:]
        result =result.strip()                                   # remove leading and trailing spaces
        street = result.replace(" ", "+")                        # replace spances with plus ( thats how the website likes it)

        # Get state name then
        state = var2['state'][i]
        state = us_state_abbrev[state]   # convert full name to abbriviation

        # get postal code.. 
        zipcode = var2['postal_code'][i]

        # get city and then replace the space with plus
        city = var2['city'][i]
        city = city.replace(" ", "+")
        print(street + " " + state + " " + zipcode +" " + city) 
    except:
        pass

SW+1st+Pl FL 33914 Cape+Coral
SW+52nd+St FL 33914 Cape+Coral
SE+10th+Ter FL 33990 Cape+Coral
Palm+Tree+Blvd FL 33904 Cape+Coral
NE+20th+Ter FL 33909 Cape+Coral
SW+17th+Pl FL 33991 Cape+Coral
NW+20th+Ter FL 33993 Cape+Coral
NW+5th+St FL 33993 Cape+Coral
NE+13th+Pl FL 33909 Cape+Coral
15th+Pl FL 33991 Cape+Coral
Sharar+Ct FL 33904 Cape+Coral
NW+Juanita+Pl FL 33993 Cape+Coral
Santa+Barbara+Blvd+N FL 33993 Cape+Coral
Yardarm+Ct FL 33914 Cape+Coral
NE+6th+St FL 33909 Cape+Coral
E+11th+Pl FL 33909 Cape+Coral
NW+10th+St FL 33993 Cape+Coral
SE+15th+Ter FL 33990 Cape+Coral
NE+20th+Ct FL 33909 Cape+Coral
SE+21st+St FL 33990 Cape+Coral
SW+35th+Ter FL 33914 Cape+Coral
SW+13th+St FL 33991 Cape+Coral
SE+24th+Pl FL 33904 Cape+Coral
SE+20th+Pl FL 33904 Cape+Coral
SW+36th+Ln FL 33914 Cape+Coral
Bayshore+Dr FL 33904 Cape+Coral
SE+16th+Pl FL 33904 Cape+Coral
NW+8th+Ter FL 33993 Cape+Coral
SE+27th+St FL 33904 Cape+Coral
6+Royal+Tee+Ct FL 33991 Cape+Coral
NE+10th+Pl FL 33909 Cape+Coral
NE+22nd+Pl FL 33909 

In [17]:
link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)
link_to_neighbors

'https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Loreto+Ct&city=Cape+Coral&state=FL&zip=33909&freeForm='

In [30]:
# Loop through all of the columns then get the names you need to make the new url to throw in the selenium code.. 
for i in range(0,len(var2)):   
    try:
        street = var2['line'][i]                                 # get the street name
        #result = ''.join([i for i in street if not i.isdigit()]) # search for digits and replace them when searching for the street
        result=street[4:]
        result =result.strip()                                   # remove leading and trailing spaces
        street = result.replace(" ", "+")                       # replace spances with plus ( thats how the website likes it)

        # Get state name then
        state = var2['state'][i]
        state = us_state_abbrev[state]   # convert full name to abbriviation

        # get postal code.. 
        zipcode = var2['postal_code'][i]

        # get city and then replace the space with plus
        city = var2['city'][i]
        city = city.replace(" ", "+")
        #print(street + " " + state + " " + zipcode +" " + city) VERIFY!!!

        # Get the url for the selenium loop... 
        link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)

        # initiate crome driver
        c = webdriver.ChromeOptions()

        # get the path to the chrome driver 
        driver = webdriver.Chrome(executable_path='C:/Users/btindol/OneDrive - Stryker/ChromeDriver/chromedriver.exe')#,options=c)

        # wait a bit
        driver.implicitly_wait(0.5)

        # use the url that got in above code 
        driver.get(link_to_neighbors) # use the url that was created before... 

        # get past the pop up alert..
        WebDriverWait(driver, 10).until(EC.alert_is_present())

        # click out of it
        driver.switch_to.alert.accept()

        # wait a bit 
        driver.implicitly_wait(3)

        # grab email to input eamil for user name 
        emailfield= driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtEmail")

        # grab password to put in pass
        passwordmessagefield = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtPassword")

        # put in email
        emailfield.send_keys(email)

        # put in password
        passwordmessagefield.send_keys(passwordz)

        # submit button 
        submitbutton = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_btnLogin")
        submitbutton.click() # send click t osubmit

        # get the excel file on the page and download.. 
        excelfile = driver.find_element_by_xpath('//*[@id="content"]/section[3]/div/div/div/span/div/div[3]/input[3]')
        excelfile.click()
    except:
        pass

In [31]:
import os
direct = 'C:\\Users\\btindol\\Downloads\\'

listz = []
listnames = []
for filename in os.listdir('C:\\Users\\btindol\\Downloads'):
    if filename.endswith(".xlsx"):
        #print(filename)
        var = direct +  filename
        print(var)
        #do your operation
        data_df = pd.read_excel(var)#, sheet_name=None)
        listnameorder=data_df['Address'][0]
        listnames.append(listnameorder)
        data_df = pd.DataFrame(data_df)
        listz.append(data_df)
        #os.remove(data_df)
listnames

C:\Users\btindol\Downloads\address_202107230558.xlsx
C:\Users\btindol\Downloads\address_202107230559 (1).xlsx
C:\Users\btindol\Downloads\address_202107230559 (2).xlsx
C:\Users\btindol\Downloads\address_202107230559.xlsx
C:\Users\btindol\Downloads\address_202107230600 (1).xlsx
C:\Users\btindol\Downloads\address_202107230600 (2).xlsx
C:\Users\btindol\Downloads\address_202107230600 (3).xlsx
C:\Users\btindol\Downloads\address_202107230600.xlsx
C:\Users\btindol\Downloads\address_202107230601 (1).xlsx
C:\Users\btindol\Downloads\address_202107230601 (2).xlsx
C:\Users\btindol\Downloads\address_202107230601 (3).xlsx
C:\Users\btindol\Downloads\address_202107230601.xlsx
C:\Users\btindol\Downloads\address_202107230602 (1).xlsx
C:\Users\btindol\Downloads\address_202107230602 (2).xlsx
C:\Users\btindol\Downloads\address_202107230602.xlsx
C:\Users\btindol\Downloads\address_202107230603 (1).xlsx
C:\Users\btindol\Downloads\address_202107230603 (2).xlsx
C:\Users\btindol\Downloads\address_202107230603.xls

['116 SE 19th Ter',
 '1701 NE 28th St',
 '18 NW 6th St',
 '101 SW 56th Ter',
 '301 SE 33rd Ter',
 '2515 SE 4th Ave',
 '431 NE 1st Pl',
 '112 Kamal Pkwy',
 '436 NE 2nd Ave',
 '11 NW 7th Ter',
 '2506 SW 22nd Pl',
 '2512 SW 28th Pl',
 '18 NW 26th Ter',
 '28 NE 16th Ter',
 '201 SW 38th St',
 '101 SE 6th St',
 '2509 SW 13th Ave',
 '617 SW 11th St',
 '2509 SW 11th Ct',
 '502 SW 27th Ter',
 '536 NE 12th St',
 '129 SE 18th Ter',
 '2 NW 14th Ave',
 '152 Destiny Cir',
 '101 SW 52nd St',
 '2421 SW 11th Ave',
 '129 SE 19th St',
 '300 NW 20th Ter',
 '2 SE 10th Ave',
 '501 SW 27th St',
 '2 SE 10th Ave',
 '221 NE 14th Ave',
 '14 NW 28th Ter',
 '101 SE 40th Ter',
 '202 SW 37th Ter',
 '216 SE 14th Ter',
 '19 NE 9th Ave',
 '2509 SW 11th Ct',
 '29 NE 15th Pl',
 '2608 SW 8th Ct',
 '4901 Seville Ct',
 '2638 2nd St',
 '5101 Rutland Ct',
 '115 SE 7th Pl',
 '5301 Bayview Ct',
 '1602 NW 30th Ln',
 '3600 NW 38th Ln',
 '2801 SE 10th Ave',
 '116 SE 19th Ter',
 '2801 SE 10th Ave',
 '201 SW 34th St',
 '2404 Santa B

In [33]:
# matching file names to the addresses.
all_addresses_cape_coral= pd.concat(listz)
all_addresses_cape_coral
#all_addresses_cape_coral.to_csv("all_cape_coral_homes.csv")

In [43]:
all_addresses_cape_coral.head()

,HouseNumber,Street,StreetSuffix,City,State,Zip,StreetName,Address,StateName,Zip4,AddressType,RBDI,Active,Fips,County,ResultCode,MAK
0,116,19th,Ter,Cape Coral,FL,33990,NaN,116 SE 19th Ter,Florida,2247.0,Street,Residential,Yes,12071.0,lee,AS01,9586730827
1,120,19th,Ter,Cape Coral,FL,33990,NaN,120 SE 19th Ter,Florida,2247.0,Street,Residential,Yes,12071.0,lee,AS01,7995891203
2,128,19th,Ter,Cape Coral,FL,33990,NaN,128 SE 19th Ter,Florida,2247.0,Street,Residential,Yes,12071.0,lee,AS01,5665627109
3,129,19th,Ter,Cape Coral,FL,33990,NaN,129 SE 19th Ter,Florida,2271.0,Street,Residential,Yes,12071.0,lee,AS01,7857484839
4,132,19th,Ter,Cape Coral,FL,33990,NaN,132 SE 19th Ter,Florida,2247.0,Street,Residential,Yes,12071.0,lee,AS01,5331981772


In [44]:
len(all_addresses_cape_coral)

21438

In [34]:
# Here you want to take the first name of the street without number so 123 house lane would be house streeet suffix lane so just put in house
def get_houses_on_street(dataframe,street):
    try:
        final= dataframe[dataframe['Street']== street]
    except:
        print("no address")
    return final

In [37]:
var = get_houses_on_street(all_addresses_cape_coral,"Amadora")
var

,HouseNumber,Street,StreetSuffix,City,State,Zip,StreetName,Address,StateName,Zip4,AddressType,RBDI,Active,Fips,County,ResultCode,MAK
0,3053,Amadora,Cir,Cape Coral,FL,33909,NaN,3053 Amadora Cir,Florida,5444.0,Street,Residential,Yes,12071.0,lee,AS01,6314001074
1,3055,Amadora,Cir,Cape Coral,FL,33909,NaN,3055 Amadora Cir,Florida,5444.0,Street,Residential,Yes,12071.0,lee,AS01,3069858079
2,3057,Amadora,Cir,Cape Coral,FL,33909,NaN,3057 Amadora Cir,Florida,5444.0,Street,Residential,Yes,12071.0,lee,AS01,8486443498
3,3059,Amadora,Cir,Cape Coral,FL,33909,NaN,3059 Amadora Cir,Florida,5444.0,Street,Residential,Yes,12071.0,lee,AS01,6246687096
4,3060,Amadora,Cir,Cape Coral,FL,33909,NaN,3060 Amadora Cir,Florida,5451.0,Street,Residential,Yes,12071.0,lee,AS01,2404021696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,3199,Amadora,Cir,Cape Coral,FL,33909,NaN,3199 Amadora Cir,Florida,5334.0,Street,Unknown,Yes,12071.0,lee,AS01,8104980123
128,3200,Amadora,Cir,Cape Coral,FL,33909,NaN,3200 Amadora Cir,Florida,5335.0,Street,Residential,Yes,12071.0,lee,AS01,7954869363
129,3201,Amadora,Cir,Cape Coral,FL,33909,NaN,3201 Amadora Cir,Florida,5337.0,Street,Residential,Yes,12071.0,lee,"AS01,AS17",3123230023
130,3203,Amadora,Cir,Cape Coral,FL,33909,NaN,3203 Amadora Cir,Florida,5337.0,Street,Residential,Yes,12071.0,lee,"AS01,AS17",1579406009


In [38]:
len(var)

132

# Use zillow to get data

In [45]:
zillow_data = ZillowWrapper("X1-ZWz1fjckjdd8gb_a2eph")

In [40]:
streetlist =[]
zestimateslist = []
bedroomlist = []
bathroomlist = []
lastsoldpricelist = []
valuationrangehighlist = []
valuationrangelowlist = []
rentzestimateamountlist = []

for i in range(0,len(var)):
    try:
        address = var['Address'][i]
        zipcode =var['Zip'][i]
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode, True)
        result = GetDeepSearchResults(deep_search_response)
        streetlist.append(result.street)
        zestimateslist.append(result.zestimate_amount)
        bedroomlist.append(result.bedrooms)
        bathroomlist.append(result.bathrooms)
        lastsoldpricelist.append(result.last_sold_price)
        valuationrangehighlist.append(result.zestimate_valuation_range_high)
        valuationrangelowlist.append(result.zestimate_valuation_range_low)
        rentzestimateamountlist.append(result.rentzestimate_amount)
        #print(i)
    except:
        #print("Error" + i)
        continue

In [66]:
for i in range(0,len(all_addresses_cape_coral)):
        print(all_addresses_cape_coral['Address'][i])

0    116 SE 19th Ter
0    1701 NE 28th St
0       18 NW 6th St
0    101 SW 56th Ter
0    301 SE 33rd Ter
          ...       
0      102 SE 5th St
0     16 NW 29th Ave
0      119 NW 3rd Pl
0     400 NW 1st Ave
0     101 SE 12th St
Name: Address, Length: 190, dtype: object
1    120 SE 19th Ter
1    1702 NE 28th St
1       26 NW 6th St
1    107 SW 56th Ter
1    303 SE 33rd Ter
          ...       
1      106 SE 5th St
1     19 NW 29th Ave
1      122 NW 3rd Pl
1     401 NW 1st Ave
1     102 SE 12th St
Name: Address, Length: 190, dtype: object
2    128 SE 19th Ter
2    1713 NE 28th St
2      103 NW 6th St
2    111 SW 56th Ter
2    307 SE 33rd Ter
          ...       
2      110 SE 5th St
2     22 NW 29th Ave
2      201 NW 3rd Pl
2     404 NW 1st Ave
2     104 SE 12th St
Name: Address, Length: 190, dtype: object
3    129 SE 19th Ter
3    1717 NE 28th St
3      122 NW 6th St
3    114 SW 56th Ter
3    310 SE 33rd Ter
          ...       
3      114 SE 5th St
3     23 NW 29th Ave
3      202 NW

KeyError: 236

In [60]:
# Do the same thing for all addresses in all_addresses_cape_coral dataframe
streetlist =[]
zestimateslist = []
bedroomlist = []
bathroomlist = []
lastsoldpricelist = []
valuationrangehighlist = []
valuationrangelowlist = []
rentzestimateamountlist = []

for i in range(0,len(all_addresses_cape_coral)):
    try:
        address = all_addresses_cape_coral['Address'][i]
        zipcode =all_addresses_cape_coral['Zip'][i]
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode, True)
        result = GetDeepSearchResults(deep_search_response)
        streetlist.append(result.street)
        print(result.street)
        zestimateslist.append(result.zestimate_amount)
#         bedroomlist.append(result.bedrooms)
#         bathroomlist.append(result.bathrooms)
#         lastsoldpricelist.append(result.last_sold_price)
#         valuationrangehighlist.append(result.zestimate_valuation_range_high)
#         valuationrangelowlist.append(result.zestimate_valuation_range_low)
#         rentzestimateamountlist.append(result.rentzestimate_amount)
#         print(i)
    except:
        #print("Error" + i)
        continue

423 SE 19th Ter
426 SE 19th Ter
427 SE 19th Ter
430 SE 19th Ter
504 SE 19th Ter
510 SE 19th Ter
601 SE 19th Ter
602 SE 19th Ter
605 SE 19th Ter
606 SE 19th Ter
609 SE 19th Ter
610 SE 19th Ter
614 SE 19th Ter
615 SE 19th Ter
619 SE 19th Ter
620 SE 19th Ter
623 SE 19th Ter
624 SE 19th Ter
628 SE 19th Ter
903 SE 19th Ter
904 SE 19th Ter
907 SE 19th Ter
908 SE 19th Ter
911 SE 19th Ter
912 SE 19th Ter
915 SE 19th Ter
916 SE 19th Ter
919 SE 19th Ter
920 SE 19th Ter
924 SE 19th Ter
927 SE 19th Ter
928 SE 19th Ter
1102 SE 19th Ter
1106 SE 19th Ter
1110 SE 19th Ter
1114 SE 19th Ter
1117 SE 19th Ter
1118 SE 19th Ter
1121 SE 19th Ter
1122 SE 19th Ter
1125 SE 19th Ter
1126 SE 19th Ter
1129 SE 19th Ter
1130 SE 19th Ter
1133 SE 19th Ter
1134 SE 19th Ter
1137 SE 19th Ter
1138 SE 19th Ter
1141 SE 19th Ter
1142 SE 19th Ter
1145 SE 19th Ter
1203 SE 19th Ter
1204 SE 19th Ter
1207 SE 19th Ter
1208 SE 19th Ter
1211 SE 19th Ter
1212 SE 19th Ter
1215 SE 19th Ter
1216 SE 19th Ter
1219 SE 19th Ter
1220 SE 19th

In [61]:
streetlist

['423 SE 19th Ter',
 '426 SE 19th Ter',
 '427 SE 19th Ter',
 '430 SE 19th Ter',
 '504 SE 19th Ter',
 '510 SE 19th Ter',
 '601 SE 19th Ter',
 '602 SE 19th Ter',
 '605 SE 19th Ter',
 '606 SE 19th Ter',
 '609 SE 19th Ter',
 '610 SE 19th Ter',
 '614 SE 19th Ter',
 '615 SE 19th Ter',
 '619 SE 19th Ter',
 '620 SE 19th Ter',
 '623 SE 19th Ter',
 '624 SE 19th Ter',
 '628 SE 19th Ter',
 '903 SE 19th Ter',
 '904 SE 19th Ter',
 '907 SE 19th Ter',
 '908 SE 19th Ter',
 '911 SE 19th Ter',
 '912 SE 19th Ter',
 '915 SE 19th Ter',
 '916 SE 19th Ter',
 '919 SE 19th Ter',
 '920 SE 19th Ter',
 '924 SE 19th Ter',
 '927 SE 19th Ter',
 '928 SE 19th Ter',
 '1102 SE 19th Ter',
 '1106 SE 19th Ter',
 '1110 SE 19th Ter',
 '1114 SE 19th Ter',
 '1117 SE 19th Ter',
 '1118 SE 19th Ter',
 '1121 SE 19th Ter',
 '1122 SE 19th Ter',
 '1125 SE 19th Ter',
 '1126 SE 19th Ter',
 '1129 SE 19th Ter',
 '1130 SE 19th Ter',
 '1133 SE 19th Ter',
 '1134 SE 19th Ter',
 '1137 SE 19th Ter',
 '1138 SE 19th Ter',
 '1141 SE 19th Ter',
 '1

In [59]:
for i in range(0,len(all_addresses_cape_coral)):
    print(all_addresses_cape_coral['Zip'][i])

0    33990
0    33909
0    33993
0    33914
0    33904
     ...  
0    33990
0    33993
0    33993
0    33993
0    33990
Name: Zip, Length: 190, dtype: int64
1    33990
1    33909
1    33993
1    33914
1    33904
     ...  
1    33990
1    33993
1    33993
1    33993
1    33990
Name: Zip, Length: 190, dtype: int64
2    33990
2    33909
2    33993
2    33914
2    33904
     ...  
2    33990
2    33993
2    33993
2    33993
2    33990
Name: Zip, Length: 190, dtype: int64
3    33990
3    33909
3    33993
3    33914
3    33904
     ...  
3    33990
3    33993
3    33993
3    33993
3    33990
Name: Zip, Length: 189, dtype: int64
4    33990
4    33909
4    33993
4    33914
4    33904
     ...  
4    33990
4    33993
4    33993
4    33993
4    33990
Name: Zip, Length: 189, dtype: int64
5    33990
5    33909
5    33993
5    33914
5    33904
     ...  
5    33990
5    33993
5    33993
5    33993
5    33990
Name: Zip, Length: 189, dtype: int64
6    33990
6    33909
6    33993
6    33914
6    339

KeyError: 236

In [51]:
len(all_addresses_cape_coral)

21438

In [54]:
street_dataframe = pd.DataFrame(
    {'streetz': streetlist,
     'zestimates': zestimateslist,
     'bedrooms': bedroomlist,
     'bathrooms': bathroomlist,
     'lastsoldprice': lastsoldpricelist,
     'valuationrangehigh': valuationrangehighlist,
     'valuationrangelowlist': valuationrangelowlist,
     'rentzestimateamountlist': rentzestimateamountlist
    })
street_dataframe

,streetz,zestimates,bedrooms,bathrooms,lastsoldprice,valuationrangehigh,valuationrangelowlist,rentzestimateamountlist
0,423 SE 19th Ter,419800,4,2.0,362000,474374,369424,2494
1,426 SE 19th Ter,333200,3,2.0,216500,356524,309876,1889
2,427 SE 19th Ter,340900,4,2.0,275000,388626,299992,2064
3,430 SE 19th Ter,241700,3,2.0,79900,265870,217530,1839
4,504 SE 19th Ter,312900,3,2.0,6000,344190,281610,2128
...,...,...,...,...,...,...,...,...
176,5336 SW 8th Ct,740800,3,2.0,471000,822288,681536,4744
177,5337 SW 8th Ct,580600,3,2.0,None,650272,516734,2600
178,5340 SW 8th Ct,1145700,3,3.0,775000,1283184,1031130,8829
179,5343 SW 8th Ct,595400,3,2.0,480000,678756,523952,2740


In [42]:
# What quantile is that house in .. 
import pandas as pd
import numpy as np
street_dataframe['zestimates'].dtypes
street_dataframe["zestimates"] = pd.to_numeric(street_dataframe["zestimates"])
street_dataframe['Quantile_rank'] = pd.qcut(street_dataframe['zestimates'], 10,labels = False)
street_dataframe

,streetz,zestimates,bedrooms,bathrooms,lastsoldprice,valuationrangehigh,valuationrangelowlist,rentzestimateamountlist,Quantile_rank
0,3053 Amadora Cir,375800.0,4,2.0,301635,417138,338220,2128,5.0
1,3055 Amadora Cir,344500.0,3,2.0,277180,392730,303160,2051,3.0
2,3057 Amadora Cir,437700.0,4,3.5,350035,485847,393930,2870,8.0
3,3059 Amadora Cir,339300.0,3,2.0,280760,383409,298584,2044,2.0
4,3060 Amadora Cir,384300.0,4,2.0,325705,403515,357399,1944,6.0
...,...,...,...,...,...,...,...,...,...
127,3199 Amadora Cir,391100.0,3,3.0,341952,441943,348079,2799,6.0
128,3200 Amadora Cir,NaN,None,None,319530,None,None,1944,NaN
129,3201 Amadora Cir,NaN,None,None,None,None,None,1927,NaN
130,3203 Amadora Cir,323300.0,4,2.0,316445,362096,287737,2132,0.0
